Extracting the backuo file to SQLite format using Docker

In [8]:
import pyodbc
import pandas as pd
import sqlite3

# 1. SQL Server connection string (adjust if needed)
sql_server_conn_str = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=localhost,1433;"
    "DATABASE=AdventureWorks;"
    "UID=SA;"
    "PWD=Ranjanmahathi@2004;"
    "TrustServerCertificate=yes;"
)

# 2. Connect to SQL Server
sql_conn = pyodbc.connect(sql_server_conn_str)
cursor = sql_conn.cursor()

# 3. Connect to SQLite
sqlite_conn = sqlite3.connect("adventureworks_exported.db")

# 4. Get all user tables
cursor.execute("""
    SELECT TABLE_SCHEMA, TABLE_NAME 
    FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_TYPE = 'BASE TABLE'
""")
tables = cursor.fetchall()

# 5. Define unsupported SQL Server data types
unsupported_types = ['hierarchyid', 'geometry', 'geography', 'sql_variant', 'xml']

# 6. Export each table
for schema, table in tables:
    full_table_name = f"{schema}.{table}"
    print(f"🔄 Exporting: {full_table_name}")

    # Get column names and types
    cursor.execute("""
        SELECT COLUMN_NAME, DATA_TYPE
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_SCHEMA = ? AND TABLE_NAME = ?
    """, (schema, table))
    
    columns = cursor.fetchall()

    # Build SELECT clause with casting
    select_columns = []
    for col_name, data_type in columns:
        if data_type.lower() in unsupported_types:
            select_columns.append(f"CAST([{col_name}] AS NVARCHAR(MAX)) AS [{col_name}]")
        else:
            select_columns.append(f"[{col_name}]")

    if not select_columns:
        print(f"⚠️ Skipping {full_table_name}: No usable columns found.")
        continue

    column_str = ", ".join(select_columns)
    query = f"SELECT {column_str} FROM [{schema}].[{table}]"

    try:
        df = pd.read_sql(query, sql_conn)
        df.to_sql(table, sqlite_conn, if_exists="replace", index=False)
        print(f"✅ Exported: {table}")
    except Exception as e:
        print(f"❌ Failed to export {table}: {e}")

# 7. Cleanup
sql_conn.close()
sqlite_conn.close()
print("🎉 Done exporting all tables to adventureworks_exported.db")

🔄 Exporting: Sales.SalesTaxRate


/var/folders/vw/qyxtp_r906j5jpy5dc2_mhpw0000gn/T/ipykernel_92947/1777130999.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, sql_conn)


✅ Exported: SalesTaxRate
🔄 Exporting: Sales.PersonCreditCard
✅ Exported: PersonCreditCard
🔄 Exporting: Person.PersonPhone
✅ Exported: PersonPhone
🔄 Exporting: Sales.SalesTerritory
✅ Exported: SalesTerritory
🔄 Exporting: Person.PhoneNumberType
✅ Exported: PhoneNumberType
🔄 Exporting: Production.Product
✅ Exported: Product
🔄 Exporting: Sales.SalesTerritoryHistory
✅ Exported: SalesTerritoryHistory
🔄 Exporting: Production.ScrapReason
✅ Exported: ScrapReason
🔄 Exporting: HumanResources.Shift
✅ Exported: Shift
🔄 Exporting: Production.ProductCategory
✅ Exported: ProductCategory
🔄 Exporting: Purchasing.ShipMethod
✅ Exported: ShipMethod
🔄 Exporting: Production.ProductCostHistory
✅ Exported: ProductCostHistory
🔄 Exporting: Production.ProductDescription
✅ Exported: ProductDescription
🔄 Exporting: Sales.ShoppingCartItem
✅ Exported: ShoppingCartItem
🔄 Exporting: Production.ProductDocument
✅ Exported: ProductDocument
🔄 Exporting: dbo.DatabaseLog
✅ Exported: DatabaseLog
🔄 Exporting: Production.Produc

/var/folders/vw/qyxtp_r906j5jpy5dc2_mhpw0000gn/T/ipykernel_92947/1777130999.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, sql_conn)


✅ Exported: ProductListPriceHistory
🔄 Exporting: Person.Address
✅ Exported: Address
🔄 Exporting: Sales.SpecialOfferProduct
✅ Exported: SpecialOfferProduct
🔄 Exporting: Production.ProductModel
✅ Exported: ProductModel
🔄 Exporting: Person.AddressType
✅ Exported: AddressType
🔄 Exporting: Person.StateProvince
✅ Exported: StateProvince
🔄 Exporting: Production.ProductModelIllustration
✅ Exported: ProductModelIllustration
🔄 Exporting: dbo.AWBuildVersion
✅ Exported: AWBuildVersion
🔄 Exporting: Production.ProductModelProductDescriptionCulture
✅ Exported: ProductModelProductDescriptionCulture
🔄 Exporting: Production.BillOfMaterials
✅ Exported: BillOfMaterials
🔄 Exporting: Sales.Store
✅ Exported: Store
🔄 Exporting: Production.ProductPhoto
✅ Exported: ProductPhoto
🔄 Exporting: Production.ProductProductPhoto
✅ Exported: ProductProductPhoto
🔄 Exporting: Production.TransactionHistory
✅ Exported: TransactionHistory
🔄 Exporting: Production.ProductReview
✅ Exported: ProductReview
🔄 Exporting: Person.Bus

In [1]:
import os

# Change to your desired directory
os.chdir("/Users/ranjanumeshrao/Downloads/DAL_Lab/Text2SQL/T2S_RAG")

# Print the current working directory to confirm
print(os.getcwd())

/Users/ranjanumeshrao/Downloads/DAL_Lab/Text2SQL/T2S_RAG


Create Chroma vector database using embedding model all-mpnet-base-v2

In [ ]:
import os
import sqlite3
import time
import shutil
import yaml
import logging
import subprocess
from datetime import datetime
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

# --- Setup Logging ---
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("T2S_RAG")

# --- Print ChromaDB and SentenceTransformers versions ---
try:
    version_info = subprocess.check_output(["pip", "show", "chromadb"]).decode()
    for line in version_info.splitlines():
        if line.startswith("Version:"):
            print(f"[INFO] ChromaDB version: {line.split(': ')[1]}")
    version_info = subprocess.check_output(["pip", "show", "sentence-transformers"]).decode()
    for line in version_info.splitlines():
        if line.startswith("Version:"):
            print(f"[INFO] SentenceTransformers version: {line.split(': ')[1]}")
except Exception as e:
    logger.warning(f"Could not get package versions: {e}")

# --- Load Config ---
config_path = "config/settings.yaml"
default_config = {
    "db_path": "data/adventureworks_exported.db",
    "persist_dir": "vector_store/adventureworks/chroma_data",
    "collection_name": "adventureworks_schema",
    "embedding_model": "all-mpnet-base-v2",
    "batch_size": 10
}

os.makedirs("config", exist_ok=True)
if os.path.exists(config_path):
    with open(config_path, "r") as f:
        config = yaml.safe_load(f) or default_config
else:
    with open(config_path, "w") as f:
        yaml.dump(default_config, f)
    config = default_config
    print(f"[INFO] Created default config at {config_path}")

db_path = config["db_path"]
persist_dir = config["persist_dir"]
collection_name = config["collection_name"]
batch_size = config["batch_size"]
embedding_model = config["embedding_model"]

# --- Ensure persist_dir exists and is writable ---
os.makedirs(persist_dir, exist_ok=True)
if not os.access(persist_dir, os.W_OK):
    raise PermissionError(f"No write permission for {persist_dir}")
print(f"[INFO] Ensured persist_dir exists and is writable: {persist_dir}")

# --- Backup existing vector DB if exists and contains files ---
if os.path.exists(persist_dir) and os.listdir(persist_dir):
    backup_dir = f"vector_store/adventureworks/backups/chroma_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(os.path.dirname(backup_dir), exist_ok=True)
    shutil.move(persist_dir, backup_dir)
    os.makedirs(persist_dir, exist_ok=True)
    print(f"[INFO] Backed up existing vector DB to: {backup_dir}")

# --- Connect to SQLite ---
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database not found: {db_path}")

try:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    print("[INFO] Connected to SQLite database")
except Exception as e:
    print(f"[ERROR] Failed to connect to SQLite database: {e}")
    raise

# --- Get table and view names ---
try:
    cursor.execute("SELECT name, type, sql FROM sqlite_master WHERE type IN ('table', 'view');")
    table_info = [(row[0], row[1], row[2]) for row in cursor.fetchall()]
    table_names = [name for name, _, _ in table_info]
    print(f"[INFO] Found {len(table_names)} objects: {', '.join(table_names)}")
except Exception as e:
    print(f"[ERROR] Failed to retrieve table/view names: {e}")
    raise

# --- Check for SalesOrderHeader ---
if "SalesOrderHeader" not in table_names:
    print("[WARN] Table 'SalesOrderHeader' not found in database")
else:
    print("[INFO] Table 'SalesOrderHeader' found in database")

# --- Build documents ---
documents = []
ids = []
metadatas = []

overview = f"Database: adventureworks\nObjects: {', '.join(table_names)}"
documents.append(overview)
ids.append("overview")
metadatas.append({"table_name": "overview", "type": "overview"})
print(f"[INFO] Added overview: {overview}")

for name, type_, sql in table_info:
    try:
        if sql:
            document = f"{type_.capitalize()}: {name}\nSchema: {sql}"
            documents.append(document)
            ids.append(f"{type_}_{name}")
            metadatas.append({"table_name": name, "type": type_})
            print(f"[SCHEMA] Added {type_}: {name}\n{document}")
        else:
            print(f"[WARN] No schema found for {type_}: {name}")
    except Exception as e:
        print(f"[ERROR] Failed to process {type_} {name}: {e}")

conn.close()
print(f"[INFO] Closed SQLite connection. Total documents prepared: {len(documents)}")

# --- Compute embeddings ---
print(f"[INFO] Loading embedding model: {embedding_model}")
try:
    model = SentenceTransformer(embedding_model, device="mps")  # Use "cuda" or "cpu" on non-Mac
    print("[OK] Embedding model loaded successfully")
except Exception as e:
    print(f"[ERROR] Failed to load embedding model: {e}")
    raise

print("[INFO] Computing embeddings...")
embeddings = []
for i in range(0, len(documents), batch_size):
    batch = documents[i:i + batch_size]
    try:
        batch_embs = model.encode(batch).tolist()
        embeddings.extend(batch_embs)
        print(f"[INFO] Computed embeddings for batch {i//batch_size + 1}: {len(batch)} documents")
    except Exception as e:
        print(f"[ERROR] Failed to compute embeddings for batch {i//batch_size + 1}: {e}")
        raise
print(f"[INFO] Generated embeddings for {len(embeddings)} documents")

# --- Initialize Chroma Client ---
try:
    client = chromadb.PersistentClient(
        path=persist_dir,
        settings=Settings(is_persistent=True),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    print(f"[INFO] Chroma client initialized with persist_dir: {persist_dir}")
except Exception as e:
    print(f"[ERROR] Failed to initialize Chroma client: {e}")
    raise

collection = client.get_or_create_collection(name=collection_name)

# --- Add documents to ChromaDB ---
print(f"[INFO] Inserting into collection '{collection_name}'...")
try:
    for i in range(0, len(documents), batch_size):
        batch_docs = documents[i:i + batch_size]
        batch_ids = ids[i:i + batch_size]
        batch_embs = embeddings[i:i + batch_size]
        batch_meta = metadatas[i:i + batch_size]
        print(f"[DEBUG] Adding batch {i//batch_size + 1}: {len(batch_docs)} documents")
        collection.add(
            documents=batch_docs,
            ids=batch_ids,
            embeddings=batch_embs,
            metadatas=batch_meta
        )
    print(f"[INFO] Added {collection.count()} documents to collection")
except Exception as e:
    print(f"[ERROR] Failed to add documents: {e}")
    raise

# --- Wait for flush ---
time.sleep(1)

# # --- Test query ---
# try:
#     test_query = "What is the schema of the SalesOrderHeader table?"
#     test_emb = model.encode([test_query]).tolist()
#     results = collection.query(
#         query_embeddings=test_emb,
#         n_results=1,
#         where={"table_name": "SalesOrderHeader"}
#     )
#     print("[TEST] Query result:", results["documents"][0][0] if results["documents"] else "No results.")
# except Exception as e:
#     print(f"[ERROR] Test query failed: {e}")

# # --- Verify SalesOrderHeader ---
# results = collection.get(ids=["table_SalesOrderHeader"])
# print("[DEBUG] SalesOrderHeader document:", results["documents"] if results["documents"] else "Not found")

# --- Verify saved files ---
print(f"[INFO] Vector DB saved to: {persist_dir}")
for root, dirs, files in os.walk(persist_dir):
    for file in files:
        rel_path = os.path.relpath(os.path.join(root, file), persist_dir)
        print(f"  - {rel_path}")

print("[INFO] Script completed successfully.")

/opt/anaconda3/envs/t2s/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] ChromaDB version: 1.0.12


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-mpnet-base-v2


[INFO] SentenceTransformers version: 4.1.0
[INFO] Ensured persist_dir exists and is writable: vector_store/adventureworks/chroma_data
[INFO] Connected to SQLite database
[INFO] Found 71 objects: SalesTaxRate, PersonCreditCard, PersonPhone, SalesTerritory, PhoneNumberType, Product, SalesTerritoryHistory, ScrapReason, Shift, ProductCategory, ShipMethod, ProductCostHistory, ProductDescription, ShoppingCartItem, ProductDocument, DatabaseLog, ProductInventory, SpecialOffer, ErrorLog, ProductListPriceHistory, Address, SpecialOfferProduct, ProductModel, AddressType, StateProvince, ProductModelIllustration, AWBuildVersion, ProductModelProductDescriptionCulture, BillOfMaterials, Store, ProductPhoto, ProductProductPhoto, TransactionHistory, ProductReview, BusinessEntity, TransactionHistoryArchive, ProductSubcategory, BusinessEntityAddress, ProductVendor, BusinessEntityContact, UnitMeasure, Vendor, ContactType, CountryRegionCurrency, CountryRegion, WorkOrder, PurchaseOrderDetail, CreditCard, Cult

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.75s/it]


...

[INFO] Computed embeddings for batch 7: 10 documents


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.29s/it]
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


[INFO] Computed embeddings for batch 8: 2 documents
[INFO] Generated embeddings for 72 documents
[INFO] Chroma client initialized with persist_dir: vector_store/adventureworks/chroma_data
[INFO] Inserting into collection 'adventureworks_schema'...
[DEBUG] Adding batch 1: 10 documents
[DEBUG] Adding batch 2: 10 documents
[DEBUG] Adding batch 3: 10 documents
[DEBUG] Adding batch 4: 10 documents
[DEBUG] Adding batch 5: 10 documents
[DEBUG] Adding batch 6: 10 documents
[DEBUG] Adding batch 7: 10 documents
[DEBUG] Adding batch 8: 2 documents
[INFO] Added 72 documents to collection


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.57s/it]

[TEST] Query result: Table: SalesOrderHeader
Schema: CREATE TABLE "SalesOrderHeader" (
"SalesOrderID" INTEGER,
  "RevisionNumber" INTEGER,
  "OrderDate" TIMESTAMP,
  "DueDate" TIMESTAMP,
  "ShipDate" TIMESTAMP,
  "Status" INTEGER,
  "OnlineOrderFlag" INTEGER,
  "SalesOrderNumber" TEXT,
  "PurchaseOrderNumber" TEXT,
  "AccountNumber" TEXT,
  "CustomerID" INTEGER,
  "SalesPersonID" REAL,
  "TerritoryID" INTEGER,
  "BillToAddressID" INTEGER,
  "ShipToAddressID" INTEGER,
  "ShipMethodID" INTEGER,
  "CreditCardID" REAL,
  "CreditCardApprovalCode" TEXT,
  "CurrencyRateID" REAL,
  "SubTotal" REAL,
  "TaxAmt" REAL,
  "Freight" REAL,
  "TotalDue" REAL,
  "Comment" TEXT,
  "rowguid" TEXT,
  "ModifiedDate" TIMESTAMP
)
[DEBUG] SalesOrderHeader document: ['Table: SalesOrderHeader\nSchema: CREATE TABLE "SalesOrderHeader" (\n"SalesOrderID" INTEGER,\n  "RevisionNumber" INTEGER,\n  "OrderDate" TIMESTAMP,\n  "DueDate" TIMESTAMP,\n  "ShipDate" TIMESTAMP,\n  "Status" INTEGER,\n  "OnlineOrderFlag" INTEGER,

Testing the vector DB

In [10]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

persist_dir = "vector_store/adventureworks/chroma_data"
collection_name = "adventureworks_schema"

client = chromadb.PersistentClient(
    path=persist_dir,
    settings=Settings(is_persistent=True),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)
collection = client.get_collection(name=collection_name)
results = collection.get(ids=["table_SalesOrderHeader"])
print("SalesOrderHeader document:", results["documents"] if results["documents"] else "Not found")

SalesOrderHeader document: ['Table: SalesOrderHeader\nSchema: CREATE TABLE "SalesOrderHeader" (\n"SalesOrderID" INTEGER,\n  "RevisionNumber" INTEGER,\n  "OrderDate" TIMESTAMP,\n  "DueDate" TIMESTAMP,\n  "ShipDate" TIMESTAMP,\n  "Status" INTEGER,\n  "OnlineOrderFlag" INTEGER,\n  "SalesOrderNumber" TEXT,\n  "PurchaseOrderNumber" TEXT,\n  "AccountNumber" TEXT,\n  "CustomerID" INTEGER,\n  "SalesPersonID" REAL,\n  "TerritoryID" INTEGER,\n  "BillToAddressID" INTEGER,\n  "ShipToAddressID" INTEGER,\n  "ShipMethodID" INTEGER,\n  "CreditCardID" REAL,\n  "CreditCardApprovalCode" TEXT,\n  "CurrencyRateID" REAL,\n  "SubTotal" REAL,\n  "TaxAmt" REAL,\n  "Freight" REAL,\n  "TotalDue" REAL,\n  "Comment" TEXT,\n  "rowguid" TEXT,\n  "ModifiedDate" TIMESTAMP\n)']


In [9]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from sentence_transformers import SentenceTransformer

persist_dir = "vector_store/adventureworks/chroma_data"
collection_name = "adventureworks_schema"

try:
    model = SentenceTransformer("all-mpnet-base-v2", device="mps")  # Use "cpu" if mps fails
    print("[INFO] Embedding model loaded")
except Exception as e:
    print(f"[ERROR] Failed to load embedding model: {e}")
    raise

try:
    client = chromadb.PersistentClient(
        path=persist_dir,
        settings=Settings(is_persistent=True),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    print("[INFO] Chroma client initialized")
except Exception as e:
    print(f"[ERROR] Failed to initialize Chroma client: {e}")
    raise

try:
    collection = client.get_collection(name=collection_name)
    print(f"Collection count: {collection.count()}")
except Exception as e:
    print(f"[ERROR] Failed to get collection: {e}")
    raise

try:
    results = collection.get(ids=["table_SalesOrderHeader"])
    print("SalesOrderHeader document:", results["documents"] if results["documents"] else "Not found")
except Exception as e:
    print(f"[ERROR] Failed to get SalesOrderHeader document: {e}")
    raise

try:
    query_text = "What is the schema of the SalesOrderHeader table?"
    query_embedding = model.encode([query_text]).tolist()
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=1,
        where={"table_name": "SalesOrderHeader"}
    )
    print("Query result:", results["documents"][0][0] if results["documents"] else "No results.")
except Exception as e:
    print(f"[ERROR] Query failed: {e}")
    raise

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-mpnet-base-v2


[INFO] Embedding model loaded
[INFO] Chroma client initialized
Collection count: 72
SalesOrderHeader document: ['Table: SalesOrderHeader\nSchema: CREATE TABLE "SalesOrderHeader" (\n"SalesOrderID" INTEGER,\n  "RevisionNumber" INTEGER,\n  "OrderDate" TIMESTAMP,\n  "DueDate" TIMESTAMP,\n  "ShipDate" TIMESTAMP,\n  "Status" INTEGER,\n  "OnlineOrderFlag" INTEGER,\n  "SalesOrderNumber" TEXT,\n  "PurchaseOrderNumber" TEXT,\n  "AccountNumber" TEXT,\n  "CustomerID" INTEGER,\n  "SalesPersonID" REAL,\n  "TerritoryID" INTEGER,\n  "BillToAddressID" INTEGER,\n  "ShipToAddressID" INTEGER,\n  "ShipMethodID" INTEGER,\n  "CreditCardID" REAL,\n  "CreditCardApprovalCode" TEXT,\n  "CurrencyRateID" REAL,\n  "SubTotal" REAL,\n  "TaxAmt" REAL,\n  "Freight" REAL,\n  "TotalDue" REAL,\n  "Comment" TEXT,\n  "rowguid" TEXT,\n  "ModifiedDate" TIMESTAMP\n)']


Batches: 100%|██████████| 1/1 [00:00<00:00,  8.68it/s]

Query result: Table: SalesOrderHeader
Schema: CREATE TABLE "SalesOrderHeader" (
"SalesOrderID" INTEGER,
  "RevisionNumber" INTEGER,
  "OrderDate" TIMESTAMP,
  "DueDate" TIMESTAMP,
  "ShipDate" TIMESTAMP,
  "Status" INTEGER,
  "OnlineOrderFlag" INTEGER,
  "SalesOrderNumber" TEXT,
  "PurchaseOrderNumber" TEXT,
  "AccountNumber" TEXT,
  "CustomerID" INTEGER,
  "SalesPersonID" REAL,
  "TerritoryID" INTEGER,
  "BillToAddressID" INTEGER,
  "ShipToAddressID" INTEGER,
  "ShipMethodID" INTEGER,
  "CreditCardID" REAL,
  "CreditCardApprovalCode" TEXT,
  "CurrencyRateID" REAL,
  "SubTotal" REAL,
  "TaxAmt" REAL,
  "Freight" REAL,
  "TotalDue" REAL,
  "Comment" TEXT,
  "rowguid" TEXT,
  "ModifiedDate" TIMESTAMP
)


In [4]:
import google.generativeai as genai
import yaml

def load_config(config_path):
    with open(config_path, 'r') as f:
        return yaml.safe_load(f)

def list_models():
    config = load_config("/Users/ranjanumeshrao/Downloads/DAL_Lab/Text2SQL/T2S_RAG/config/settings.yaml")
    genai.configure(api_key=config['gemini']['api_key'])
    models = genai.list_models()
    for model in models:
        print(f"Model: {model.name}, Supported Methods: {model.supported_generation_methods}")

if __name__ == "__main__":
    list_models()

Model: models/embedding-gecko-001, Supported Methods: ['embedText', 'countTextTokens']
Model: models/gemini-1.0-pro-vision-latest, Supported Methods: ['generateContent', 'countTokens']
Model: models/gemini-pro-vision, Supported Methods: ['generateContent', 'countTokens']
Model: models/gemini-1.5-pro-latest, Supported Methods: ['generateContent', 'countTokens']
Model: models/gemini-1.5-pro-001, Supported Methods: ['generateContent', 'countTokens', 'createCachedContent']
Model: models/gemini-1.5-pro-002, Supported Methods: ['generateContent', 'countTokens', 'createCachedContent']
Model: models/gemini-1.5-pro, Supported Methods: ['generateContent', 'countTokens']
Model: models/gemini-1.5-flash-latest, Supported Methods: ['generateContent', 'countTokens']
Model: models/gemini-1.5-flash-001, Supported Methods: ['generateContent', 'countTokens', 'createCachedContent']
Model: models/gemini-1.5-flash-001-tuning, Supported Methods: ['generateContent', 'countTokens', 'createTunedModel']
Model: m

In [8]:
import os
os.chdir(path="/Users/ranjanumeshrao/Downloads/DAL_Lab/Text2SQL/T2S_RAG")

In [9]:
! pwd

/Users/ranjanumeshrao/Downloads/DAL_Lab/Text2SQL/T2S_RAG


In [10]:
import os
import sqlite3
import time
import shutil
import yaml
import logging
import subprocess
from datetime import datetime
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

# --- Setup Logging ---
logging.basicConfig(level=logging.DEBUG)  # Changed to DEBUG for detailed output
logger = logging.getLogger("T2S_RAG")

# --- Print ChromaDB and SentenceTransformers versions ---
try:
    version_info = subprocess.check_output(["pip", "show", "chromadb"]).decode()
    for line in version_info.splitlines():
        if line.startswith("Version:"):
            print(f"[INFO] ChromaDB version: {line.split(': ')[1]}")
    version_info = subprocess.check_output(["pip", "show", "sentence-transformers"]).decode()
    for line in version_info.splitlines():
        if line.startswith("Version:"):
            print(f"[INFO] SentenceTransformers version: {line.split(': ')[1]}")
except Exception as e:
    logger.warning(f"Could not get package versions: {e}")

# --- Load Config ---
config_path = "config/settings.yaml"
default_config = {
    "db_path": "data/adventureworks_exported.db",
    "persist_dir": "vector_store/adventureworks/chroma_data",
    "collection_name": "adventureworks_schema",
    "embedding_model": "all-mpnet-base-v2",
    "batch_size": 10
}

os.makedirs("config", exist_ok=True)
if os.path.exists(config_path):
    with open(config_path, "r") as f:
        config = yaml.safe_load(f) or default_config
else:
    with open(config_path, "w") as f:
        yaml.dump(default_config, f)
    config = default_config
    print(f"[INFO] Created default config at {config_path}")

db_path = config["db_path"]
persist_dir = config["persist_dir"]
collection_name = config["collection_name"]
batch_size = config["batch_size"]
embedding_model = config["embedding_model"]

# --- Ensure persist_dir exists and is writable ---
os.makedirs(persist_dir, exist_ok=True)
if not os.access(persist_dir, os.W_OK):
    raise PermissionError(f"No write permission for {persist_dir}")
print(f"[INFO] Ensured persist_dir exists and is writable: {persist_dir}")

# --- Backup existing vector DB if exists and contains files ---
if os.path.exists(persist_dir) and os.listdir(persist_dir):
    backup_dir = f"vector_store/adventureworks/backups/chroma_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(os.path.dirname(backup_dir), exist_ok=True)
    shutil.move(persist_dir, backup_dir)
    os.makedirs(persist_dir, exist_ok=True)
    print(f"[INFO] Backed up existing vector DB to: {backup_dir}")

# --- Connect to SQLite ---
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database not found: {db_path}")

try:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    print("[INFO] Connected to SQLite database")
except Exception as e:
    print(f"[ERROR] Failed to connect to SQLite database: {e}")
    raise

# --- Get table and view names ---
try:
    cursor.execute("SELECT name, type, sql FROM sqlite_master WHERE type IN ('table', 'view');")
    table_info = [(row[0], row[1], row[2]) for row in cursor.fetchall()]
    table_names = [name for name, _, _ in table_info]
    print(f"[INFO] Found {len(table_names)} objects: {', '.join(table_names)}")
except Exception as e:
    print(f"[ERROR] Failed to retrieve table/view names: {e}")
    raise

# --- Check for SalesOrderHeader ---
if "SalesOrderHeader" not in table_names:
    print("[WARN] Table 'SalesOrderHeader' not found in database")
else:
    print("[INFO] Table 'SalesOrderHeader' found in database")

# --- Build documents ---
documents = []
ids = []
metadatas = []

overview = f"Database: adventureworks\nObjects: {', '.join(table_names)}"
documents.append(overview)
ids.append("overview")
metadatas.append({"table_name": "overview", "type": "overview"})
print(f"[INFO] Added overview: {overview}")

for name, type_, sql in table_info:
    try:
        if sql:
            document = f"{type_.capitalize()}: {name}\nSchema: {sql}"
            documents.append(document)
            ids.append(f"{type_}_{name}")
            metadatas.append({"table_name": name, "type": type_})
            print(f"[SCHEMA] Added {type_}: {name}\n{document}")
        else:
            print(f"[WARN] No schema found for {type_}: {name}")
    except Exception as e:
        print(f"[ERROR] Failed to process {type_} {name}: {e}")

conn.close()
print(f"[INFO] Closed SQLite connection. Total documents prepared: {len(documents)}")

# --- Check for unique IDs ---
if len(ids) != len(set(ids)):
    print(f"[ERROR] Duplicate IDs found: {ids}")
else:
    print(f"[DEBUG] All IDs are unique")

# --- Compute embeddings ---
print(f"[INFO] Loading embedding model: {embedding_model}")
try:
    model = SentenceTransformer(embedding_model, device="mps")  # Use "cuda" or "cpu" on non-Mac
    print("[OK] Embedding model loaded successfully")
except Exception as e:
    print(f"[ERROR] Failed to load embedding model: {e}")
    raise

print("[INFO] Computing embeddings...")
embeddings = []
for i in range(0, len(documents), batch_size):
    batch = documents[i:i + batch_size]
    try:
        batch_embs = model.encode(batch).tolist()
        embeddings.extend(batch_embs)
        print(f"[INFO] Computed embeddings for batch {i//batch_size + 1}: {len(batch)} documents")
    except Exception as e:
        print(f"[ERROR] Failed to compute embeddings for batch {i//batch_size + 1}: {e}")
        raise

# --- Check embedding shapes ---
if embeddings:
    print(f"[DEBUG] First embedding shape: {len(embeddings[0])} dimensions")
    embedding_lengths = [len(emb) for emb in embeddings]
    if len(set(embedding_lengths)) == 1:
        print(f"[DEBUG] All embeddings have {embedding_lengths[0]} dimensions")
    else:
        print(f"[ERROR] Embeddings have varying dimensions: {embedding_lengths}")
print(f"[INFO] Generated embeddings for {len(embeddings)} documents")

# --- Ensure all lists have the same length ---
assert len(documents) == len(ids) == len(embeddings) == len(metadatas), "Mismatch in list lengths"

# --- Initialize Chroma Client ---
try:
    client = chromadb.PersistentClient(
        path=persist_dir,
        settings=Settings(is_persistent=True),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    print(f"[INFO] Chroma client initialized with persist_dir: {persist_dir}")
except Exception as e:
    print(f"[ERROR] Failed to initialize Chroma client: {e}")
    raise

collection = client.get_or_create_collection(name=collection_name)
print(f"[DEBUG] Initial collection count: {collection.count()}")

# --- Add documents to ChromaDB ---
print(f"[INFO] Inserting into collection '{collection_name}'...")
try:
    for i in range(0, len(documents), batch_size):
        batch_docs = documents[i:i + batch_size]
        batch_ids = ids[i:i + batch_size]
        batch_embs = embeddings[i:i + batch_size]
        batch_meta = metadatas[i:i + batch_size]
        assert len(batch_docs) == len(batch_ids) == len(batch_embs) == len(batch_meta), f"Mismatch in batch {i//batch_size + 1}"
        print(f"[DEBUG] Adding batch {i//batch_size + 1}: {len(batch_docs)} documents with IDs: {batch_ids}")
        collection.add(
            documents=batch_docs,
            ids=batch_ids,
            embeddings=batch_embs,
            metadatas=batch_meta
        )
        print(f"[DEBUG] Batch {i//batch_size + 1} added. Current collection count: {collection.count()}")
    print(f"[INFO] Added {collection.count()} documents to collection")
except Exception as e:
    print(f"[ERROR] Failed to add documents: {e}")
    raise

# --- Wait for flush ---
time.sleep(1)

# # --- Test query ---
# try:
#     test_query = "What is the schema of the SalesOrderHeader table?"
#     test_emb = model.encode([test_query]).tolist()
#     results = collection.query(
#         query_embeddings=test_emb,
#         n_results=1,
#         where={"table_name": "SalesOrderHeader"}
#     )
#     print("[TEST] Query result:", results["documents"][0][0] if results["documents"] else "No results.")
# except Exception as e:
#     print(f"[ERROR] Test query failed: {e}")

# # --- Verify SalesOrderHeader ---
# results = collection.get(ids=["table_SalesOrderHeader"])
# print("[DEBUG] SalesOrderHeader document:", results["documents"] if results["documents"] else "Not found")

# --- Verify saved files ---
print(f"[INFO] Vector DB saved to: {persist_dir}")
for root, dirs, files in os.walk(persist_dir):
    for file in files:
        rel_path = os.path.relpath(os.path.join(root, file), persist_dir)
        print(f"  - {rel_path}")

print("[INFO] Script completed successfully.")

[INFO] ChromaDB version: 0.5.5


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-mpnet-base-v2
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443


[INFO] SentenceTransformers version: 2.7.0
[INFO] Ensured persist_dir exists and is writable: vector_store/adventureworks/chroma_data
[INFO] Connected to SQLite database
[INFO] Found 71 objects: SalesTaxRate, PersonCreditCard, PersonPhone, SalesTerritory, PhoneNumberType, Product, SalesTerritoryHistory, ScrapReason, Shift, ProductCategory, ShipMethod, ProductCostHistory, ProductDescription, ShoppingCartItem, ProductDocument, DatabaseLog, ProductInventory, SpecialOffer, ErrorLog, ProductListPriceHistory, Address, SpecialOfferProduct, ProductModel, AddressType, StateProvince, ProductModelIllustration, AWBuildVersion, ProductModelProductDescriptionCulture, BillOfMaterials, Store, ProductPhoto, ProductProductPhoto, TransactionHistory, ProductReview, BusinessEntity, TransactionHistoryArchive, ProductSubcategory, BusinessEntityAddress, ProductVendor, BusinessEntityContact, UnitMeasure, Vendor, ContactType, CountryRegionCurrency, CountryRegion, WorkOrder, PurchaseOrderDetail, CreditCard, Cult

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/modules.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/README.md HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/modules.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/sentence_bert_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-v2/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-mpnet-base-

[OK] Embedding model loaded successfully
[INFO] Computing embeddings...


...

DEBUG:chromadb.config:Starting component System
DEBUG:chromadb.config:Starting component Posthog
DEBUG:chromadb.config:Starting component OpenTelemetryClient
DEBUG:chromadb.config:Starting component SqliteDB
DEBUG:chromadb.config:Starting component QuotaEnforcer
DEBUG:chromadb.config:Starting component LocalSegmentManager
DEBUG:chromadb.config:Starting component SegmentAPI
DEBUG:chromadb.config:Starting component PersistentLocalHnswSegment


[INFO] Chroma client initialized with persist_dir: vector_store/adventureworks/chroma_data
[DEBUG] Initial collection count: 0
[INFO] Inserting into collection 'adventureworks_schema'...
[DEBUG] Adding batch 1: 10 documents with IDs: ['overview', 'table_SalesTaxRate', 'table_PersonCreditCard', 'table_PersonPhone', 'table_SalesTerritory', 'table_PhoneNumberType', 'table_Product', 'table_SalesTerritoryHistory', 'table_ScrapReason', 'table_Shift']
[DEBUG] Batch 1 added. Current collection count: 10
[DEBUG] Adding batch 2: 10 documents with IDs: ['table_ProductCategory', 'table_ShipMethod', 'table_ProductCostHistory', 'table_ProductDescription', 'table_ShoppingCartItem', 'table_ProductDocument', 'table_DatabaseLog', 'table_ProductInventory', 'table_SpecialOffer', 'table_ErrorLog']
[DEBUG] Batch 2 added. Current collection count: 20
[DEBUG] Adding batch 3: 10 documents with IDs: ['table_ProductListPriceHistory', 'table_Address', 'table_SpecialOfferProduct', 'table_ProductModel', 'table_Addr

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): us.i.posthog.com:443


[INFO] Vector DB saved to: vector_store/adventureworks/chroma_data
  - chroma.sqlite3
  - 974fb79e-1e8b-498c-8ae2-13d2019c3d3f/data_level0.bin
  - 974fb79e-1e8b-498c-8ae2-13d2019c3d3f/length.bin
  - 974fb79e-1e8b-498c-8ae2-13d2019c3d3f/link_lists.bin
  - 974fb79e-1e8b-498c-8ae2-13d2019c3d3f/header.bin
[INFO] Script completed successfully.


DEBUG:urllib3.connectionpool:https://us.i.posthog.com:443 "POST /batch/ HTTP/1.1" 200 15
